Credit=Siddarthan
https://www.youtube.com/watch?v=nacLBdyG6jE&t=2767s

About the Dataset:

I have made some changes


1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

In [ ]:
!unzip '/content/fake-news.zip'

Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

Data Pre-processing

In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv')
test=pd.read_csv('test.csv')

In [ ]:
news_dataset.shape

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')
test=test.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
test['content'] = test['author']+' '+test['title']

In [ ]:
print(news_dataset['content'])

In [ ]:
test.head()

In [ ]:
test.drop(columns=['text','author','title'],inplace=True)

In [ ]:
test.head()

In [ ]:
news_dataset.drop(columns=['text','author','title'],inplace=True)

In [ ]:
news_dataset.head()

In [ ]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
X1 = test['content'].values


In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
Y.shape

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
X1 = vectorizer.transform(test['content'].values)


In [ ]:
print(X1)

In [ ]:
print(X)

Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
print(X_train)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

Evaluation

accuracy score

In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

In [ ]:
test_prediction = model.predict(X1)

In [ ]:
submission_df = pd.DataFrame({
    'id': test['id'],
    'label': test_prediction
})
submission_df.to_csv('submission.csv', index=False)